In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# add . to module name
import sys
sys.path.append('../src/')

In [41]:
from package.logger import setup
setup("INFO")

In [42]:
import pandas as pd
import os
from package.logger import llog, Timed
from package import storage, osm
from package.footpaths import get_osm_path_from_city_id, download_osm, create_nearby_stops_map, create_nx_graph, add_nearest_node_to_stops, create_i_graph, create_footpaths


In [43]:
city_id = "Koeln"
osm_path = get_osm_path_from_city_id(city_id)
stops_path = "../data/cleaned/stops.csv"

if not os.path.exists(osm_path) and city_id:
	llog.info("Downloading OSM data")
	download_osm(city_id, osm_path)
else:
	llog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Reading OSM network"):
	# TODO: optimally we should first crop the data and then read the network
	nodes, edges = osm.read_network(osm_reader)

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

with Timed.info("Cropping OSM network to stops"):
	nodes, edges = osm.crop_to_stops(nodes, edges, stops_df)


[14:29:31] INFO     Using existing OSM data                                                         ]8;id=599627;file:///tmp/ipykernel_69596/4199287156.py\4199287156.py]8;;\:]8;id=16925;file:///tmp/ipykernel_69596/4199287156.py#9\9]8;;\

[14:29:32] INFO     Reading OSM network                                                            ]8;id=397684;file:///tmp/ipykernel_69596/4199287156.py\4199287156.py]8;;\:]8;id=336427;file:///tmp/ipykernel_69596/4199287156.py#13\13]8;;\

In [27]:
s_edges = edges[['u', 'v', 'length']]

In [38]:
import mcr_py
mcr_py.read_edges_improved(s_edges.to_dict('records'))

[{'u': 21063145, 'v': 7151289920, 'length': 7.161},
 {'u': 7151289920, 'v': 10929975, 'length': 7.575},
 {'u': 10929975, 'v': 10853912, 'length': 132.615},
 {'u': 388516, 'v': 2160088528, 'length': 25.665},
 {'u': 367149, 'v': 1672380543, 'length': 7.73},
 {'u': 1672380543, 'v': 1588847134, 'length': 11.032},
 {'u': 1588847134, 'v': 1352634, 'length': 48.349},
 {'u': 10989096, 'v': 10989200, 'length': 14.338},
 {'u': 10989200, 'v': 10990037, 'length': 91.626},
 {'u': 10990037, 'v': 68732822, 'length': 54.729},
 {'u': 68732822, 'v': 1672368051, 'length': 15.542},
 {'u': 1672368051, 'v': 1672368053, 'length': 7.816},
 {'u': 1672368053, 'v': 1672380621, 'length': 7.298},
 {'u': 1672380621, 'v': 10989051, 'length': 5.31},
 {'u': 2276506373, 'v': 2276506377, 'length': 11.568},
 {'u': 2276506377, 'v': 313720425, 'length': 57.668},
 {'u': 313720425, 'v': 2276506375, 'length': 116.652},
 {'u': 359887, 'v': 2234629647, 'length': 7.942},
 {'u': 2234629647, 'v': 9719444415, 'length': 28.645},
 {'

In [22]:
avg_walking_speed = 1.4 # m/s
max_walking_duration = 10*60 # s

In [30]:
with Timed.info("Finding potential nearby stops for each stop"):
	nearby_stops_map = create_nearby_stops_map(
		stops_df, avg_walking_speed, max_walking_duration
	)
with Timed.info("Creating networkx graph"):
	nx_graph = create_nx_graph(osm_reader, nodes, edges)

with Timed.info("Adding nearest network node to each stop"):
	stops_df = add_nearest_node_to_stops(stops_df, nx_graph)

with Timed.info("Creating igraph graph"):
	i_graph = create_i_graph(osm_reader, nodes, edges)


[11:23:50] INFO     Finding potential nearby stops for each stop                                    ]8;id=756141;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=588964;file:///tmp/ipykernel_14256/2151612084.py#1\1]8;;\

           INFO     Finding potential nearby stops for each stop done (0.16 seconds)                ]8;id=492692;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=489664;file:///tmp/ipykernel_14256/2151612084.py#1\1]8;;\

           INFO     Creating networkx graph                                                         ]8;id=838587;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=239768;file:///tmp/ipykernel_14256/2151612084.py#5\5]8;;\

[11:23:54] INFO     Creating networkx graph done (4.27 seconds)                                     ]8;id=429943;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=136170;file:///tmp/ipykernel_14256/2151612084.py#5\5]8;;\

           INFO     Adding nearest network node to each stop                                        ]8;id=953157;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=717446;file:///tmp/ipykernel_14256/2151612084.py#8\8]8;;\

           INFO     Adding nearest network node to each stop done (0.15 seconds)                    ]8;id=611680;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=100671;file:///tmp/ipykernel_14256/2151612084.py#8\8]8;;\

           INFO     Creating igraph graph                                                          ]8;id=916875;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=529276;file:///tmp/ipykernel_14256/2151612084.py#11\11]8;;\

[11:23:57] INFO     Creating igraph graph done (2.41 seconds)                                      ]8;id=232065;file:///tmp/ipykernel_14256/2151612084.py\2151612084.py]8;;\:]8;id=592691;file:///tmp/ipykernel_14256/2151612084.py#11\11]8;;\

In [31]:
with Timed.info("Calculating distances between nearby stops"):
	footpaths = create_footpaths(
		i_graph,
		stops_df,
		nearby_stops_map,
		avg_walking_speed,
	)


           INFO     Calculating distances between nearby stops                                      ]8;id=394148;file:///tmp/ipykernel_14256/3660572610.py\3660572610.py]8;;\:]8;id=461670;file:///tmp/ipykernel_14256/3660572610.py#1\1]8;;\

  0%|          | 0/97 [00:00<?, ?it/s]

[11:23:59] INFO     Calculating distances between nearby stops done (2.66 seconds)                  ]8;id=461512;file:///tmp/ipykernel_14256/3660572610.py\3660572610.py]8;;\:]8;id=686527;file:///tmp/ipykernel_14256/3660572610.py#1\1]8;;\

In [38]:
s = stops_df.set_index("stop_id")
s.loc[["6","7"]]

,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,geometry,nearest_node,nearest_node_dist
stop_id,,,,,,,,,,,,,
6,,Köln Rathaus,,50.938467,6.959697,,,0,,,POINT (6.95970 50.93847),8479332252,2.213592
7,,Köln Appellhofplatz,,50.939972,6.950437,,,0,,,POINT (6.95044 50.93997),25630718,4.600852


In [32]:
footpaths

{'6': {'7': 618, '8': 496, '9': 534, '752': 477},
 '7': {'6': 618,
  '8': 594,
  '9': 648,
  '30': 644,
  '32': 712,
  '752': 418,
  '971': 322},
 '8': {'6': 496,
  '7': 594,
  '9': 93,
  '36': 811,
  '653': 791,
  '752': 186,
  '971': 645,
  '7394': 685},
 '9': {'6': 534,
  '7': 648,
  '8': 93,
  '35': 825,
  '36': 842,
  '653': 822,
  '752': 280,
  '7394': 716},
 '30': {'7': 644, '31': 480, '32': 392, '867': 436, '971': 500},
 '31': {'30': 480, '32': 597, '236': 493, '756': 774, '867': 66},
 '32': {'7': 712, '30': 392, '31': 597, '867': 562, '971': 476},
 '34': {'35': 486, '37': 236, '305': 708, '313': 656, '653': 484, '7394': 519},
 '35': {'9': 825,
  '34': 486,
  '36': 387,
  '37': 668,
  '38': 613,
  '653': 2,
  '7394': 461},
 '36': {'8': 811, '9': 842, '35': 387, '38': 448, '653': 385},
 '37': {'34': 236, '35': 668, '313': 420, '318': 603, '653': 666, '7394': 598},
 '38': {'35': 613,
  '36': 448,
  '305': 493,
  '311': 578,
  '312': 294,
  '653': 611,
  '750': 680,
  '826': 738,


In [18]:
df_edges = pd.DataFrame(edges).drop(columns=["geometry"])

In [21]:
nodes.to_csv("./tmp/nodes.csv", index=False)
edges.to_csv("./tmp/edges.csv", index=False)